# CoSim - One Snapshot

In [1]:
import numpy as np

import andes
import ams

import pandas as pd

import json

import cvxpy as cp

In [2]:
ams.__version__

'0.6.4.post49.dev0+g2685857'

In [3]:
andes.config_logger(stream_level=20)

In [4]:
sa = andes.load(andes.get_case('ieee14/ieee14_ace.xlsx'),
                setup=True,
                no_output=True,
                )


Working directory: "/Users/jinningwang/Documents/work/ams/dev/demo"
> Loaded config from file "/Users/jinningwang/.andes/andes.rc"
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Parsing input file "/Users/jinningwang/Documents/work/mambaforge/envs/ams/lib/python3.9/site-packages/andes/cases/ieee14/ieee14_ace.xlsx"...
Input file parsed in 0.1313 seconds.
System internal structure set up in 0.0208 seconds.


In [5]:
ams.config_logger(stream_level=10)

In [6]:
sa.PFlow.run()

-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: Off
   Sparse solver: KLU
 Solution method: NR method
Power flow initialized in 0.0041 seconds.
0: |F(x)| = 0.5605182134
1: |F(x)| = 0.006202200332
2: |F(x)| = 5.819382825e-06
3: |F(x)| = 6.957087684e-12
Converged in 4 iterations in 0.0179 seconds.


True

In [7]:
sp = ams.load(ams.get_case('ieee14/ieee14.xlsx'),
              setup=True,
              )

Working directory: "/Users/jinningwang/Documents/work/ams/dev/demo"
Input format guessed as xlsx.
Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/ieee14/ieee14.xlsx"...
Input file parsed in 0.0280 seconds.
Adjusting the bus index to start from 0.
System set up in 0.0023 seconds.


In [8]:
sa = sp.to_andes(setup=False,
                 addfile=andes.get_case('ieee14/ieee14_wt3.xlsx'),
                 overwrite=True,
                 keep=False,
                 no_output=True,
                 )

sa.add("Toggle", dict(model='SynGen', dev="GENROU_4", t=11.0))

sa.setup()

JSON file written to "ieee14.json"
System convert to ANDES in 0.1016 seconds, save to "ieee14.json".
Working directory: "/Users/jinningwang/Documents/work/ams/dev/demo"
> Loaded config from file "/Users/jinningwang/.andes/andes.rc"
> Reloaded generated Python code of module "pycode".
Parsing input file "ieee14.json"...
Input file parsed in 0.0018 seconds.
Converted file is removed. Set "keep = True" to keep it.
Addfile format guessed as xlsx.
Parsing additional file "/Users/jinningwang/Documents/work/mambaforge/envs/ams/lib/python3.9/site-packages/andes/cases/ieee14/ieee14_wt3.xlsx"...
Power flow models exist in the addfile. Only dynamic models will be kept.
Correct SynGen idx to match StaticGen.
Correct RenGen idx to match StaticGen.
Addfile parsed in 0.0539 seconds.
AMS system 0x2814d3e20 link to ANDES system 0x2814eb5e0 in 0.0453 seconds.
System internal structure set up in 0.0225 seconds.


True

In [9]:
sp.ACOPF.run()

Setup model for ACOPF
- Generating symbols for ACOPF
Set constrs power balance: self.constrs["pb"]=cp.sum(self.routine.pd.v) - cp.sum(self.pg) == 0
ACOPF model set up in 0.0028 seconds.
Adjusting the bus index to start from 0.
PYPOWER Version 5.1.4, 27-June-2018
 -- AC Optimal Power Flow



Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011


ACOPF completed in 0.2610 seconds with exit code 0.


Converged!


0

In [10]:
sp.ACOPF.pg

Var: StaticGen.pg, a=[28 29 30 31 32], v=[0.49996655 0.10000065 0.10000064 0.10000065 1.49809555]

In [11]:
sp.dyn.send()

Sending model online status to ANDES...
Sending ACOPF results to ANDES...


True

In [12]:
sa.PFlow.run()

-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: Off
   Sparse solver: KLU
 Solution method: NR method
Power flow initialized in 0.0038 seconds.
0: |F(x)| = 0.697239972
1: |F(x)| = 0.03123382496
2: |F(x)| = 9.221449594e-05
3: |F(x)| = 8.563380977e-10
Converged in 4 iterations in 0.0035 seconds.


True

In [13]:
idx_tg = sa.TGOV1.idx.v
vmax0 = sa.TGOV1.get(src='VMAX', attr='v', idx=idx_tg)
sa.TGOV1.set(src='VMAX', attr='v', idx=idx_tg, value=10 * vmax0)
vmin0 = sa.TGOV1.get(src='VMIN', attr='v', idx=idx_tg)
sa.TGOV1.set(src='VMIN', attr='v', idx=idx_tg, value=np.zeros_like(vmin0))

True

In [14]:
sa.TDS.config.tf = 10
sa.TDS.run()


-> Time Domain Simulation Summary:
Sparse Solver: KLU
Simulation time: 0.0-10 s.
Fixed step size: h=33.33 ms. Shrink if not converged.
Initialization for dynamics completed in 0.0556 seconds.
Initialization was successful.


  0%|          | 0/100 [00:00<?, ?%/s]

<Toggle Toggler_1>: Line.Line_1 status changed to 0 at t=1.0 sec.
<Toggle Toggler_2>: Line.Line_1 status changed to 1 at t=1.1 sec.


Simulation to t=10.00 sec completed in 1.2327 seconds.


True

In [15]:
sa.TDS.config.tf = 20
sa.TDS.run()

  0%|          | 0/100 [00:00<?, ?%/s]

<Toggle Toggle_3>: SynGen.GENROU_4 status changed to 0 at t=11.0 sec.


Simulation to t=20.00 sec completed in 1.8054 seconds.


True

In [16]:
sp.dyn.send()

Sending model online status to ANDES...
Sending ACOPF results to ANDES...


True

In [17]:
sp.dyn.receive()

Receiving model status from ANDES...


True